In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Get the data
df_train = pd.read_table("crime-train.txt")
df_test = pd.read_table("crime-test.txt")

y = df_train["ViolentCrimesPerPop"].values
y_test = df_test["ViolentCrimesPerPop"].values
x = df_train.drop("ViolentCrimesPerPop", axis = 1).values
x_test = df_test.drop("ViolentCrimesPerPop", axis = 1).values
(n,d) = x_train.shape
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)
idx1 = df_train.drop("ViolentCrimesPerPop", axis = 1).columns.get_loc("agePct12t29")
idx2 = df_train.drop("ViolentCrimesPerPop", axis = 1).columns.get_loc("pctWSocSec")
idx3 = df_train.drop("ViolentCrimesPerPop", axis = 1).columns.get_loc("pctUrban")
idx4 = df_train.drop("ViolentCrimesPerPop", axis = 1).columns.get_loc("agePct65up")
idx5 = df_train.drop("ViolentCrimesPerPop", axis = 1).columns.get_loc("householdsize")
idx = [idx1, idx2, idx3, idx4, idx5]
print(idx)

In [ ]:
def CDA_better(x, y, lam, delta = 10e-8):
    going = True
    (n,d) = x.shape
    w = np.ones((d,))
    w_prev = np.copy(w)
    count = 0
    while going:
        b = np.mean(y - w@x.T)
        a = 2*np.sum(np.abs(x)**2,axis=0)
        for k in range(d):
            x_new = np.append(x[:,:k], x[:,k+1:], axis = 1)
            w_new = np.append(w[:k], w[k+1:])
            ck = 2 * np.sum(x[:,k] @ (y - (b + w_new@x_new.T)))
            if(ck < -lam):
                w[k] = (ck+lam)/(a[k])
            elif (ck>lam):
                w[k] = (ck-lam)/(a[k])
            else:
                w[k] = 0
                
                
        #Check if converged
        diff = np.max(np.abs(w - w_prev))
        if( np.max(np.abs(w - w_prev)) < delta):
            return w
        count += 1
        
        w_prev = np.copy(w)

In [ ]:
def find_lam_max(x, y, d):
    lam_max_list = np.zeros(d)
    for k in range(d):
        summation = 0
        for i in range(n):
            summation += x[i,k] * (y[i] - np.mean(y))*2
        lam_max_list[k] = summation
    return np.max(lam_max_list)


In [ ]:
def error(truth, pred):
    mse = np.mean((truth - pred)**2)
    return mse

In [ ]:
scale_fac = 2
num_nonzero_list = []
lam_list = []
mse_train_list = []
mse_test_list = []
weights = []


lam = find_lam_max(x, y, d)
print(lam)

i = 0
num_nonzero = 0
while lam > 0.01:
    print(i)
    i += 1
    w = CDA_better(x, y, lam, delta = 1e-3)
    
    pred_train = w@x.T
    pred_test = w@x_test.T
    mse_train_list.append(error(y, pred_train))
    mse_test_list.append(error(y_test, pred_test))
    
    weights.append(w[idx])
    
    num_nonzero= np.count_nonzero(w)
    num_nonzero_list.append(num_nonzero)
    lam_list.append(lam)
    lam = lam/scale_fac
    
    print("num_nonzero_list = ", num_nonzero_list)
    print("lam_list = ", lam_list)
    print("weights = ", weights)
    print("mse_train_list = ", mse_train_list)
    print("mse_test_list = ", mse_test_list)



In [ ]:
print(lam_list)
print(num_nonzero_list)
plt.plot(lam_list, num_nonzero_list)
plt.xlabel(r'$\lambda$')
plt.ylabel('Number of nonzero terms')
plt.xscale('log')
plt.title('Number of nonzero terms as regularization changes')

In [ ]:
weights_np = np.array(weights)
print(weights_np.shape)
plt.plot(lam_list, weights_np[:,0], label = "agePCt12t29")
plt.plot(lam_list, weights_np[:,1], label = "pctWSocSec")
plt.plot(lam_list, weights_np[:,2], label = "pctUrban")
plt.plot(lam_list, weights_np[:,3], label = "agePCT65Up")
plt.plot(lam_list, weights_np[:,4], label = "householsize")
plt.legend()
plt.xlabel(r'$\lambda$')
plt.ylabel('Weight term')
plt.xscale('log')
plt.title('Regularization paths for select variables')

In [ ]:
plt.plot(lam_list, mse_train_list, label = "Training")
plt.plot(lam_list, mse_test_list, label = "Testing")
plt.xlabel(r'$\lambda$')
plt.legend()
plt.ylabel('Mean Squared Error')
plt.xscale('log')
plt.title('Mean Squared Error as regularization changes')
# plt.yscale('log')

In [ ]:
#running for lambda = 30 for question d
w = CDA_better(x, y, 30, delta = 1e-3)
print(w)
print("Max coefficient")
print(np.argmax(w))
print(df_train.drop("ViolentCrimesPerPop", axis = 1).columns[np.argmax(w)])
print("Min coefficient")
print(np.argmin(w))
print(df_train.drop("ViolentCrimesPerPop", axis = 1).columns[np.argmin(w)])